# Victor Manuel Gómez Espinosa
## CIMAT Unidad Monterrey
### Análisis temporal de los tópicos de Riesgos a la Alza y Baja de 2011-2020

# Librerías

In [1]:
#instalacion de librerias
!pip install fasttext
!pip install sklearn_extensions

     |████████████████████████████████| 71kB 4.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093670 sha256=67d1ddf105c3748467507fb4a573f097924eb1148e89d3c933e947ba58a07d4b
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
  Created wheel for sklearn-extensions: filename=sklearn_extensions-0.0.2-py2.py3-none-any.whl size=24578 sha256=7bec7e2dabae9264f65c3d126ab4bde403a7324e089d1f4a745854f9e30bcb06
  Stored in directory: /root/.cache/pip/wheels/b5/26/b2/09100728168b45e4de834b943a588028744fdfaa9ac979ffb2
Successfully built sklearn-extensions


In [2]:
#librerias
import os
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
#from gensim.models.keyedvectors import KeyedVectors
import fasttext.util
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA,KernelPCA
import seaborn as sns
import plotly.express as px
plt.style.use('ggplot')
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopSpanish=set(stopwords.words('spanish'))
from sklearn_extensions.fuzzy_kmeans import FuzzyKMeans
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from ipywidgets import interact
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.



In [3]:
os.chdir('/content/drive/My Drive/Colab') #cambiar la ubicacion a la carpeta donde se encuentra este archivo
!ls #listar los documentos en esta ruta

 1_EstadísticasRiesgos.ipynb		  ITAER.xlsx
 2_ClustersRiesgos.ipynb		  NO.xlsx
 3_AnalisisTemporalTopicosRiesgos.ipynb   PIB.xlsx
 4_IndiceVsIndicadores.ipynb		  RiesgosLimpia.xlsx
 CE.xlsx				  Spanish
 clusters.xlsx				  SUR.xlsx
 CN.xlsx				 'word_vectors_fastTEXT (1).bin'
 General.xlsx				  word_vectors_fastTEXT.bin


# Datos

In [4]:
#leer datos
df=pd.read_excel('RiesgosLimpia.xlsx', index_col=0, parse_dates=True )
df.head() #primeros datos

,region,rama,riesgo,texto,TextsLen,TopicCluster
trimestre,,,,,,
1T11,CE,Agropecuario-Industria Alimentaria,Alza,"La empresa, ya que se dedica a hacer construcc...",36,2
1T11,CE,Agropecuario-Industria Alimentaria,Alza,El que los consumidores gocen de liquidez impu...,42,2
1T11,CE,Agropecuario-Industria Alimentaria,Alza,Se requiere incrementar el poder adquisitivo d...,31,2
1T11,CE,Comercio,Alza,Considera que una nueva ley fiscal podría prop...,16,2
1T11,CE,Agropecuario-Industria Alimentaria,Alza,"Demanda EUA, materias primas e inseguridad",7,2


In [5]:
df.info() #informacion del df

<class 'pandas.core.frame.DataFrame'>
Index: 35895 entries, 1T11 to 4T20
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        35895 non-null  object
 1   rama          35895 non-null  object
 2   riesgo        35895 non-null  object
 3   texto         35895 non-null  object
 4   TextsLen      35895 non-null  int64 
 5   TopicCluster  35895 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
TrainDF=df.loc['1T11':'3T20']
TrainDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35482 entries, 1T11 to 3T20
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        35482 non-null  object
 1   rama          35482 non-null  object
 2   riesgo        35482 non-null  object
 3   texto         35482 non-null  object
 4   TextsLen      35482 non-null  int64 
 5   TopicCluster  35482 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [7]:
#ramas
ramas=np.unique(df.rama.values)
ramas=list(ramas)
ramas.insert(0,'Todas')
ramas

['Todas',
 'Actividad Económica en General',
 'Agropecuario-Industria Alimentaria',
 'Comercio',
 'Construcción e Inmuebles',
 'Construcción e inmuebles',
 'Manufacturas',
 'Minería, Electricidad, Gas y Agua',
 'Minería, electricidad, gas y agua',
 'Otros servicios',
 'Servicios Financieros y Seguros',
 'Transporte y Comunicaciones',
 'Turismo']

In [8]:
#trimestres
trimestres=np.unique(df.index.values)
nt=[trimestres[i][0] for i in range(trimestres.shape[0])]#trimestre
year=[trimestres[i][-2:] for i in range(trimestres.shape[0])] #año
trimDict={'trimestre': trimestres,
          'numero': nt,
          'año':year}

df_trimestres=pd.DataFrame(trimDict).sort_values(by=['año','numero'])
trim_ord=df_trimestres.trimestre.values
trim_ord

array(['1T11', '2T11', '3T11', '4T11', '1T12', '2T12', '3T12', '4T12',
       '1T13', '2T13', '3T13', '4T13', '1T14', '2T14', '3T14', '4T14',
       '1T15', '2T15', '3T15', '4T15', '1T16', '2T16', '3T16', '4T16',
       '1T17', '2T17', '3T17', '4T17', '1T18', '2T18', '3T18', '4T18',
       '1T19', '2T19', '3T19', '4T19', '1T20', '2T20', '3T20', '4T20'],
      dtype=object)

In [9]:
#corpus
CorpusTr=TrainDF.texto.values


# Vocabulario documentos

In [10]:
#vocabulario

n_words = 25000
tokenizer = Tokenizer(num_words=n_words) # el Tokenizador solo tomará las 'n_words' palabras más frecuentes
tokenizer.fit_on_texts(CorpusTr) 

# word_index contiene un diccionario con el vocabulario y sus indices en orden ascendente de acuerdo a su frecuencia
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
word_counts = tokenizer.word_counts #frecuencias

Found 14475 unique tokens.


# Embeddings (FastText)

https://fasttext.cc/docs/en/python-module.html#important-preprocessing-data-encoding-conventions

In [11]:
#carga los embbegings
ft = fasttext.load_model('word_vectors_fastTEXT.bin')

In [12]:
#funcion para obtener los embeddings del vocabulario (todos)
def filter_FT(vocab_dict=word_index, ft_model=ft, NotInVocab_zeros=True, wordvec_dim=300, stopSp=stopSpanish):
  embedding_matrix=np.zeros((len(vocab_dict)+1, wordvec_dim)) #matriz inicial
  vocabFT=ft_model.words #vocabulario FastText
  NotInVocab=0 #contador de palabras fuera de vocabulario de FastTExt
  words=0 #contador de palabras a mostrar

  for word, i in vocab_dict.items():
    #recorre los elementos del diccionario del vocabulario de los documentos
    if word in vocabFT:
      #si esta en FastText
      embedding_vector=ft_model.get_word_vector(word)
    else:
      embedding_vector=None
      NotInVocab+=1
    if embedding_vector is not None: #si esta en FastText
      embedding_matrix[i]=embedding_vector
    else:
      words+=1
      if words<=2:
        print(word)
      if NotInVocab_zeros==False: #si no se van a poner 0's
        embedding_matrix[i]=ft_model.get_word_vector(word)

  print(NotInVocab)  
  return embedding_matrix

In [13]:
embedding_matrix=filter_FT(NotInVocab_zeros=False) #True con zeros en las palabras fuera de vocabulario, palabras fuera de vocabulario 1286, muestra las primeras 3 fuera de vocabulario
embedding_matrix.shape

covid
tmec
1280


(14476, 300)

# Reducción de dimensión, clustering y visualización por trimestre

## Train

In [14]:
# reducción de dimension para poder visualizar
X_train=embedding_matrix[1:,:]
pca = KernelPCA(n_components=2, kernel='cosine').fit(X_train)
X_transformedTr = pca.transform(X_train)

In [15]:
# guarda los resultados en un df
pca_dataset = pd.DataFrame(X_transformedTr)
pca_dataset.columns=['c1','c2']
pca_dataset['Words']=word_index.keys()

In [16]:
# asigna a un cluster
fuzzy_kmeans = FuzzyKMeans(k=3)
fuzzy_kmeans.fit(X_transformedTr)
pca_dataset['Labels']= fuzzy_kmeans.labels_ #etiquetas
pt=fuzzy_kmeans.fuzzy_labels_ #probabilidades
centroids=fuzzy_kmeans.cluster_centers_ #centroides
centroids.shape

(3, 2)

In [17]:
#funcion para obtener todos los datos para el trimestre seleccionado
def TrimAnalyzer(trim):
  
  TestDF=df.loc[trim] #interesantes 4T13, 4T20
  CorpusT=TestDF.texto.values #textos
  X_test_seq=tokenizer.texts_to_sequences(CorpusT) #obtiene las secuencias

  # promedio de embbedings por documento
  X_test=np.zeros((len(X_test_seq), 300)) #matriz inicial
  embb=np.zeros((300,)) #matriz inicial
  i=0
  for seq in X_test_seq:
    n=len(seq)
    if n<1:
      print(i, seq, CorpusT[i])
    embbi=embb
    for j in seq:
      embbi[:]=embbi[:]+embedding_matrix[j,:]
    X_test[i,:]=embbi[:]/n
    i+=1

  X_transformedT = pca.transform(X_test) #proyecta

  #cluster, el centroide más cercano
  nbrs = NearestNeighbors(n_neighbors=1).fit(centroids)
  _, cl = nbrs.kneighbors(X_transformedT) 

  
  TestDF['TopicCluster']= cl #etiquetas

  return TestDF, X_transformedT

# Serie temporal

In [18]:
#asigna un cluster a cada documento
clusters=[]
for trim in trim_ord:
  TestDF, _=TrimAnalyzer(trim)
  cluster=list(TestDF['TopicCluster'].values)
  clusters.extend(cluster)

clusters=np.array(clusters)
df['TopicCluster']=clusters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [19]:
df.to_excel('clusters'+'.xlsx')

In [20]:
def sentiment(trim, df=df): #sentimiento o riesgo, por teorema de la probabilidad total
  dftrim=df.loc[trim].copy()
  dfcluster=dftrim.TopicCluster.value_counts()/len(dftrim)
  p=0.0
  n=0.0
  pw=[0.492445		,0.444444,0.708526] #para los clusters 0 y 2, los pesos son las medias de las frecuencias. para el cluster 1 son el minimo (alza) y maximo (baja)
  nw=[0.507555	,0.555556	,0.291474		]

  topicsdist=[]
  for i in range(0,3):
    try: 
      clw=dfcluster[i]
    except:
      clw=0.0
    p+=pw[i]*clw # probabilidad total
    n+=nw[i]*clw
    sentimentp=p #p/(p+n+1e-6)
    sentimentn=1-sentimentp

    topicsdist.append(clw)  
  return sentimentn, sentimentp, topicsdist #devuelde la probabilidad que el trimestre sea riesgo a la baja dado las frecuencias de los documentos en los clusters (centrada en 0), frecuencias de cada cluster

In [27]:
def plotTimeSeries(filter):
  saveFile=True
  TestDF=df.copy() 
  list0=[]
  list1=[]
  list2=[]
  listsent=[]
  if filter=='General':
    filtered=TestDF
  else:
    filtered=TestDF[ (TestDF.region==filter) ]
  
  
  for trim in trim_ord:
    sentimentn,  sentimentp, (cl0,cl1,cl2)=sentiment(trim, df=filtered)
    if filter=='SUR' or filter=='General':
      sentiment_index =sentimentn # a la baja
    else:
      sentiment_index =sentimentp # al alza

    list0.append(cl0)
    list1.append(cl1)
    list2.append(cl2)
    listsent.append(sentiment_index)
  
  dfdict={#'date':trim_ord,
          'CL0':list0,
          'CL1':list1,
          'CL2':list2,
          'Index':listsent}
  df_ts=pd.DataFrame(dfdict)
  df_ts.index=trim_ord
  df_ts.index.name='Trimestre'

  min=np.amin(df_ts.Index.values)
  max=np.amax(df_ts.Index.values)
  minidx=df_ts[df_ts.Index==min].index
  maxidx=df_ts[df_ts.Index==max].index
  
  if saveFile==True:
    if filter=='General':
      df_ts.loc['2T13':'3T20'].to_excel(filter+'.xlsx')
    else:
      df_ts.loc['2T13':'2T20'].to_excel(filter+'.xlsx')


    
  plt.figure(figsize=(10,10))
  df_ts.plot(subplots=True,style='.-',figsize=(10,10))
  plt.axhline(y=0.5,color='blue',linestyle='--')
  #plt.axhspan(0.0,0.01,color='green',alpha=0.2)
  #plt.axhspan(0.0,-0.01,color='red',alpha=0.2)
  plt.title(filter)

  plt.show()
  print('min ',minidx[0])
  print()
  print('max ',maxidx[0])
    

In [28]:
interact(plotTimeSeries,filter=['General','NO','SUR','CN','CE']) #tarda un poco...

interactive(children=(Dropdown(description='filter', options=('General', 'NO', 'SUR', 'CN', 'CE'), value='Gene…

<function __main__.plotTimeSeries>

## Observaciones:
- 1T11-4T12 Calderon. 2T11-1T13 sin registros a la alza en los datos proporcionados.
- 1T13-4T18 Peña
- 1T19-4T20 AMLO

## Test

In [23]:
def PlotProy(trim):
  TestDF, X_transformedT=TrimAnalyzer(trim)
  #grafico
  fig = px.scatter(pca_dataset, x='c1', y='c2', color='Labels', hover_data=['Words'], opacity=0.15)
  # plot the centroids
  fig.add_trace( go.Scatter(
              x=X_transformedT[:, 0],
              y=X_transformedT[:, 1],
              mode="markers",
              
              
              name='Documentos',
              hovertemplate =
              
              
              '<b>%{text}</b>',
              text = TestDF.texto.values #textos,
              
              
              #marker=dict(
              
              #size=[20,20,20],
              #color=['black', 'black', 'black']
              
              #)
          
            
          ) )
  fig.update_layout(
      autosize=False,
      width=800,
      height=800,
  )
  fig.show()

In [24]:
interact(PlotProy,trim=trim_ord) #tarda un poco...

interactive(children=(Dropdown(description='trim', options=('1T11', '2T11', '3T11', '4T11', '1T12', '2T12', '3…

<function __main__.PlotProy>

# Tópicos más frecuentes en el cluster por trimestre

In [25]:
#funcion para hacer la nube de las 50 palabras mas frecuentes
def WordCloudImg(Corpus,name,maxword=10):
    
    #junta todos los textos
    prep = " ".join(review for review in Corpus if len(review)>3 and review.isnumeric()==False )
    my_stopwords=set(STOPWORDS)
    my_stopwords.update(stopSpanish)
    my_stopwords.update(['zee'])
    cloudtxt=WordCloud(background_color='white',stopwords=my_stopwords, max_words=maxword).generate(prep)
    plt.figure(figsize=(10,10))
    plt.imshow(cloudtxt,interpolation='bilinear')
    plt.title(str(maxword)+' Palabras más frecuentes en cluster: '+name)
    plt.axis('off')
    plt.show()
    cloudtxt.to_file(name+".png")

def plotimg(trim,Cluster,riesgo,region,ramas,maxword=10): 
    TestDF, _=TrimAnalyzer(trim) 
    if Cluster=='0':
        cl=0
    elif Cluster=='1':
        cl=1
    elif Cluster=='2':
        cl=2

    
    #selecciona las 3 diferentes
    if riesgo!='Ambos' and region!='Todos' and ramas!='Todas': 
      filtered=TestDF[(TestDF.TopicCluster==cl) & (TestDF.riesgo==riesgo) & (TestDF.region	==region)& (TestDF.rama	==ramas)]
    
    #selecciona 2 diferentes
    elif riesgo!='Ambos' and region!='Todos' and ramas=='Todas': 
      filtered=TestDF[(TestDF.TopicCluster==cl) & (TestDF.riesgo==riesgo) & (TestDF.region	==region)]

    elif riesgo!='Ambos' and region=='Todos' and ramas!='Todas': 
      filtered=TestDF[(TestDF.TopicCluster==cl) & (TestDF.riesgo==riesgo) & (TestDF.rama	==ramas)]

    elif riesgo=='Ambos' and region!='Todos' and ramas=='Todas': 
      filtered=TestDF[(TestDF.TopicCluster==cl) & (TestDF.region	==region)]

    #selecciona 1 diferente
    elif riesgo!='Ambos' and region=='Todos' and ramas=='Todas': 
      filtered=TestDF[(TestDF.TopicCluster==cl) & (TestDF.riesgo==riesgo) ]
    elif region!='Todos' and riesgo=='Ambos'   and ramas=='Todas':
      filtered=TestDF[(TestDF.TopicCluster==cl) &  (TestDF.region	==region)]
    elif ramas!='Todas' and riesgo=='Ambos' and region=='Todos'  :
      filtered=TestDF[(TestDF.TopicCluster==cl) &  (TestDF.rama	==ramas)]
    
    #ambos, Todos, Todas
    else:
      filtered=TestDF[(TestDF.TopicCluster==cl) ]

    Textos=filtered['texto'].values #documentos
    Corpus=[]
    for texto in Textos:
      palabras=word_tokenize(texto)
      Corpus.extend(palabras)

    print('Corpus size: '+str(len(Textos)))
    print('Tokens: '+str(len(Corpus)))
    if len(Corpus)>=1:
      WordCloudImg(Corpus,Cluster,maxword)
    else:
      print('No hay textos en este cluster')

In [26]:
interact(plotimg,trim=trim_ord,Cluster=['0', '1', '2'],riesgo=['Ambos','Alza','Baja'],region=['Todos','NO','SUR','CN','CE'],ramas=ramas,maxword=[10,25,50]) #tarda un poco...

interactive(children=(Dropdown(description='trim', options=('1T11', '2T11', '3T11', '4T11', '1T12', '2T12', '3…

<function __main__.plotimg>